**Pre-conv**

In [4]:
#PUt the paths here.
test_data = '/content/drive/MyDrive/Brain Cancer/Test Data for RPi'
original_model = '/content/drive/MyDrive/Brain Cancer/v3/PreConv.h5'
tflite_model = '/content/drive/MyDrive/Brain Cancer/v3/PreConv_TFLite.tflite'
tflite_model_quantized = '/content/drive/MyDrive/Brain Cancer/v3/PreConv_Quantized.tflite'


#Imort the packages
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(tf.keras.__version__)    
#Output must be 
#2.3.0
#2.4.0
import numpy as np
import cv2
import  os
from sklearn.preprocessing import OneHotEncoder
import pywt
print('All packages are imported')


def read_image(im_path):
  img = cv2.imread(im_path,0)
  img = cv2.resize(img,(240,240))
  return img
  
  
time0 = time.time()
x_test, y_test = [], []
list_folder=os.listdir(path = test_data)
for i in list_folder:
  new_path=os.path.join(test_data,i) 
  pic_list=os.listdir(new_path)                                              
  for img in pic_list:  
    im_path = os.path.join(new_path,img)
    image = read_image(im_path)
    if i == 'flair':
      label = 0
    elif i == 't1':
      label = 1
    elif i == 't1ce':
      label = 2
    elif i == 't2':
      label = 3
    x_test.append(image) 
    y_test.append(label) 


X_test = np.array(x_test).reshape(-1,240,240,1)
X_test = X_test/255
y_test = np.array(y_test).reshape(-1,1)
encoder = OneHotEncoder()
y_test = encoder.fit_transform(y_test)
Y_test = y_test.toarray()
time1 = time.time()
print("Test images reading is done")
print("Number of Test images",X_test.shape[0])
print("Time for reading and Preprocessing", time1 - time0)
print("Average time for reading and pre-processing", (time1-time0)/X_test.shape[0])


#Read the models
# Initialize TFLite interpreter using the model.
# Load TFLite model and allocate tensors.
time2 = time.time()
#original
original = tf.keras.models.load_model(original_model)
#TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model)
interpreter.allocate_tensors()
#quantized
interpreter_q = tf.lite.Interpreter(model_path=tflite_model_quantized)
interpreter_q.allocate_tensors()
time3 = time.time()
print("\nTime for Loading all models", time3 - time2)


# Get input and output tensors.
#TFLite
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
#Quantized
input_details_q = interpreter_q.get_input_details()
output_details_q = interpreter_q.get_output_details()

#Evaluation
#original
time4 = time.time()
evaluation = original.evaluate(X_test, Y_test)
time5 = time.time()
print('\Accuracy of Original : ', evaluation[1])
print("Total time to predict all", time5 -time4)
print("Average time to predict: ", (time5 -time4)/len(X_test))

time6 = time.time()
n = 0  #count for accuracy
normal = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter.set_tensor(input_details[0]['index'], test_image)
  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  
  if np.argmax(output_data)==np.argmax(Y_test[i]):
    n+=1
  else:
    normal.append(i)

time7 = time.time()
print('\n\nAccuracy of TFLite model= ', n/len(X_test))
print("Time to predict all " , len(X_test), " images is: ", time7-time6)
print("Average time to predict a image is: ", (time7-time6)/len(X_test))
    
time8 = time.time()
q = 0
quantized = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter_q.set_tensor(input_details_q[0]['index'], test_image)
  interpreter_q.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data_q = interpreter_q.get_tensor(output_details_q[0]['index'])


  if np.argmax(output_data_q)==np.argmax(Y_test[i]):
    q+=1
  else:
    quantized.append(i)

time9 = time.time()
print('\n\nAccuracy of quantized model= ', q/len(X_test))
print("Time to predict all ", len(X_test), " images is: ", time9-time8)
print("Average time to predict a image is: ", (time9-time8)/len(X_test))

2.4.0
2.4.0
All packages are imported
Test images reading is done
Number of Test images 193
Time for reading and Preprocessing 0.6117165088653564
Average time for reading and pre-processing 0.00316951558997594

Time for Loading all models 0.1615772247314453
7/7 [==============================] - 0s 8ms/step - loss: 0.5516 - accuracy: 0.9534
\Accuracy of Original :  0.9533678889274597
Total time to predict all 0.249525785446167
Average time to predict:  0.0012928797173376529


Accuracy of TFLite model=  0.9481865284974094
Time to predict all  193  images is:  1.429603099822998
Average time to predict a image is:  0.00740726994726942


Accuracy of quantized model=  0.9430051813471503
Time to predict all  193  images is:  11.35194206237793
Average time to predict a image is:  0.05881835265480793


**DWT + Pre-Conv**

In [5]:
#PUt the paths here.
test_data = '/content/drive/MyDrive/Brain Cancer/Test Data for RPi'
original_model = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Com_+_Pre-Conv.h5'
tflite_model = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Com_+_Pre-Conv_TFLite.tflite'
tflite_model_quantized = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Com_+_Pre-Conv_Quantized.tflite'


#Imort the packages
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(tf.keras.__version__)    
#Output must be 
#2.3.0
#2.4.0
import numpy as np
import cv2
import  os
from sklearn.preprocessing import OneHotEncoder
import pywt
print('All packages are imported')


x_test, y_test = [], []
list_folder=os.listdir(path = test_data)
for i in list_folder:
  new_path=os.path.join(test_data,i) 
  pic_list=os.listdir(new_path)                                              
  for img in pic_list:  
    im_path = os.path.join(new_path,img)
    image = read_image(im_path)
    cA, _ = pywt.dwt2(image,'db2')    #Performing DWT operation
    if i == 'flair':
      label = 0
    elif i == 't1':
      label = 1
    elif i == 't1ce':
      label = 2
    elif i == 't2':
      label = 3
    x_test.append(cA) 
    y_test.append(label) 


X_test = np.array(x_test).reshape(-1,121,121,1)
X_test = X_test/255
y_test = np.array(y_test).reshape(-1,1)
encoder = OneHotEncoder()
y_test = encoder.fit_transform(y_test)
Y_test = y_test.toarray()
time1 = time.time()
print("Test images reading is done")
print("Number of Test images",X_test.shape[0])
print("Time for reading and Preprocessing", time1 - time0)
print("Average time for reading and pre-processing", (time1-time0)/X_test.shape[0])


#Read the models
# Initialize TFLite interpreter using the model.
# Load TFLite model and allocate tensors.
time2 = time.time()
#original
original = tf.keras.models.load_model(original_model)
#TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model)
interpreter.allocate_tensors()
#quantized
interpreter_q = tf.lite.Interpreter(model_path=tflite_model_quantized)
interpreter_q.allocate_tensors()
time3 = time.time()
print("\nTime for Loading all models", time3 - time2)


# Get input and output tensors.
#TFLite
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
#Quantized
input_details_q = interpreter_q.get_input_details()
output_details_q = interpreter_q.get_output_details()

#Evaluation
#original
time4 = time.time()
evaluation = original.evaluate(X_test, Y_test)
time5 = time.time()
print('\Accuracy of Original : ', evaluation[1])
print("Total time to predict all", time5 -time4)
print("Average time to predict: ", (time5 -time4)/len(X_test))

time6 = time.time()
n = 0  #count for accuracy
normal = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter.set_tensor(input_details[0]['index'], test_image)
  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  
  if np.argmax(output_data)==np.argmax(Y_test[i]):
    n+=1
  else:
    normal.append(i)

time7 = time.time()
print('\n\nAccuracy of TFLite model= ', n/len(X_test))
print("Time to predict all " , len(X_test), " images is: ", time7-time6)
print("Average time to predict a image is: ", (time7-time6)/len(X_test))
    
time8 = time.time()
q = 0
quantized = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter_q.set_tensor(input_details_q[0]['index'], test_image)
  interpreter_q.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data_q = interpreter_q.get_tensor(output_details_q[0]['index'])


  if np.argmax(output_data_q)==np.argmax(Y_test[i]):
    q+=1
  else:
    quantized.append(i)

time9 = time.time()
print('\n\nAccuracy of quantized model= ', q/len(X_test))
print("Time to predict all ", len(X_test), " images is: ", time9-time8)
print("Average time to predict a image is: ", (time9-time8)/len(X_test))

2.4.0
2.4.0
All packages are imported
Test images reading is done
Number of Test images 193
Time for reading and Preprocessing 14.739645719528198
Average time for reading and pre-processing 0.07637122134470568

Time for Loading all models 0.18595075607299805
7/7 [==============================] - 0s 5ms/step - loss: 0.3583 - accuracy: 0.9223
\Accuracy of Original :  0.9222797751426697
Total time to predict all 0.2420654296875
Average time to predict:  0.0012542250242875647


Accuracy of TFLite model=  0.917098445595855
Time to predict all  193  images is:  0.46111631393432617
Average time to predict a image is:  0.0023892036991415865


Accuracy of quantized model=  0.917098445595855
Time to predict all  193  images is:  2.989502429962158
Average time to predict a image is:  0.015489649896176986


**Pre-Conv + Spark**

In [6]:
#PUt the paths here.
test_data = '/content/drive/MyDrive/Brain Cancer/Test Data for RPi'
original_model = '/content/drive/MyDrive/Brain Cancer/v3/PreConv_Spark.h5'
tflite_model = '/content/drive/MyDrive/Brain Cancer/v3/PreConv_Spark_TFLite.tflite'
tflite_model_quantized = '/content/drive/MyDrive/Brain Cancer/v3/PreConv_Spark_Quantized.tflite'


#Imort the packages
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(tf.keras.__version__)    
#Output must be 
#2.3.0
#2.4.0
import numpy as np
import cv2
import  os
from sklearn.preprocessing import OneHotEncoder
print('All packages are imported')


def read_image(im_path):
  img = cv2.imread(im_path,0)
  img = cv2.resize(img,(240,240))
  return img
  
  
time0 = time.time()
x_test, y_test = [], []
list_folder=os.listdir(path = test_data)
for i in list_folder:
  new_path=os.path.join(test_data,i) 
  pic_list=os.listdir(new_path)                                              
  for img in pic_list:  
    im_path = os.path.join(new_path,img)
    image = read_image(im_path)
    if i == 'flair':
      label = 0
    elif i == 't1':
      label = 1
    elif i == 't1ce':
      label = 2
    elif i == 't2':
      label = 3
    x_test.append(image) 
    y_test.append(label) 


X_test = np.array(x_test).reshape(-1,240,240,1)
X_test = X_test/255
y_test = np.array(y_test).reshape(-1,1)
encoder = OneHotEncoder()
y_test = encoder.fit_transform(y_test)
Y_test = y_test.toarray()
time1 = time.time()
print("Test images reading is done")
print("Number of Test images",X_test.shape[0])
print("Time for reading and Preprocessing", time1 - time0)
print("Average time for reading and pre-processing", (time1-time0)/X_test.shape[0])


#Read the models
# Initialize TFLite interpreter using the model.
# Load TFLite model and allocate tensors.
time2 = time.time()
#original
original = tf.keras.models.load_model(original_model)
#TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model)
interpreter.allocate_tensors()
#quantized
interpreter_q = tf.lite.Interpreter(model_path=tflite_model_quantized)
interpreter_q.allocate_tensors()
time3 = time.time()
print("\nTime for Loading all models", time3 - time2)


# Get input and output tensors.
#TFLite
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
#Quantized
input_details_q = interpreter_q.get_input_details()
output_details_q = interpreter_q.get_output_details()

#Evaluation
#original
time4 = time.time()
evaluation = original.evaluate(X_test, Y_test)
time5 = time.time()
print('\Accuracy of Original : ', evaluation[1])
print("Total time to predict all", time5 -time4)
print("Average time to predict: ", (time5 -time4)/len(X_test))

time6 = time.time()
n = 0  #count for accuracy
normal = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter.set_tensor(input_details[0]['index'], test_image)
  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  
  if np.argmax(output_data)==np.argmax(Y_test[i]):
    n+=1
  else:
    normal.append(i)

time7 = time.time()
print('\n\nAccuracy of TFLite model= ', n/len(X_test))
print("Time to predict all " , len(X_test), " images is: ", time7-time6)
print("Average time to predict a image is: ", (time7-time6)/len(X_test))
    
time8 = time.time()
q = 0
quantized = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter_q.set_tensor(input_details_q[0]['index'], test_image)
  interpreter_q.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data_q = interpreter_q.get_tensor(output_details_q[0]['index'])


  if np.argmax(output_data_q)==np.argmax(Y_test[i]):
    q+=1
  else:
    quantized.append(i)

time9 = time.time()
print('\n\nAccuracy of quantized model= ', q/len(X_test))
print("Time to predict all ", len(X_test), " images is: ", time9-time8)
print("Average time to predict a image is: ", (time9-time8)/len(X_test))


2.4.0
2.4.0
All packages are imported
Test images reading is done
Number of Test images 193
Time for reading and Preprocessing 0.6022207736968994
Average time for reading and pre-processing 0.0031203148896212403

Time for Loading all models 0.18079876899719238
7/7 [==============================] - 0s 15ms/step - loss: 0.6870 - accuracy: 0.9067
\Accuracy of Original :  0.9067357778549194
Total time to predict all 0.36591219902038574
Average time to predict:  0.0018959181296393044


Accuracy of TFLite model=  0.9015544041450777
Time to predict all  193  images is:  0.8111746311187744
Average time to predict a image is:  0.004202977363309712


Accuracy of quantized model=  0.8911917098445595
Time to predict all  193  images is:  2.3139142990112305
Average time to predict a image is:  0.011989193259125546


**Spark**

In [7]:
#PUt the paths here.
test_data = '/content/drive/MyDrive/Brain Cancer/Test Data for RPi'
original_model = '/content/drive/MyDrive/Brain Cancer/v3/spark.h5'
tflite_model = '/content/drive/MyDrive/Brain Cancer/v3/spark_TFLite.tflite'
tflite_model_quantized = '/content/drive/MyDrive/Brain Cancer/v3/spark_Quantized.tflite'


#Imort the packages
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(tf.keras.__version__)    
#Output must be 
#2.3.0
#2.4.0
import numpy as np
import cv2
import  os
from sklearn.preprocessing import OneHotEncoder
print('All packages are imported')

def read_image(im_path):
  img = cv2.imread(im_path,0)
  img = cv2.resize(img,(240,240))
  return img
  
  
time0 = time.time()
x_test, y_test = [], []
list_folder=os.listdir(path = test_data)
for i in list_folder:
  new_path=os.path.join(test_data,i) 
  pic_list=os.listdir(new_path)                                              
  for img in pic_list:  
    im_path = os.path.join(new_path,img)
    image = read_image(im_path)
    if i == 'flair':
      label = 0
    elif i == 't1':
      label = 1
    elif i == 't1ce':
      label = 2
    elif i == 't2':
      label = 3
    x_test.append(image) 
    y_test.append(label) 

print("Test images reading is done")
X_test = np.array(x_test).reshape(-1,240,240,1)
X_test = X_test/255
y_test = np.array(y_test).reshape(-1,1)
encoder = OneHotEncoder()
y_test = encoder.fit_transform(y_test)
Y_test = y_test.toarray()
time1 = time.time()
print("Test images reading is done")
print("Number of Test images",X_test.shape[0])
print("Time for reading and Preprocessing", time1 - time0)
print("Average time for reading and pre-processing", (time1-time0)/X_test.shape[0])


#Read the models
# Initialize TFLite interpreter using the model.
# Load TFLite model and allocate tensors.
time2 = time.time()
#original
original = tf.keras.models.load_model(original_model)
#TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model)
interpreter.allocate_tensors()
#quantized
interpreter_q = tf.lite.Interpreter(model_path=tflite_model_quantized)
interpreter_q.allocate_tensors()
time3 = time.time()
print("\nTime for Loading all models", time3 - time2)


# Get input and output tensors.
#TFLite
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
#Quantized
input_details_q = interpreter_q.get_input_details()
output_details_q = interpreter_q.get_output_details()

#Evaluation
#original
time4 = time.time()
evaluation = original.evaluate(X_test, Y_test)
time5 = time.time()
print('\nAccuracy of Original : ', evaluation[1])
print("Total time to predict all", time5 -time4)
print("Average time to predict", (time5 -time4)/len(X_test))

time6 = time.time()
n = 0  #count for accuracy
normal = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter.set_tensor(input_details[0]['index'], test_image)
  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  
  if np.argmax(output_data)==np.argmax(Y_test[i]):
    n+=1
  else:
    normal.append(i)

time7 = time.time()
print('\n\nAccuracy of TFLite model= ', n/len(X_test))
print("Time to predict all " , len(X_test), " images is: ", time7-time6)
print("Average time to predict a image is: ", (time7-time6)/len(X_test))
    
time8 = time.time()
q = 0
quantized = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter_q.set_tensor(input_details_q[0]['index'], test_image)
  interpreter_q.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data_q = interpreter_q.get_tensor(output_details_q[0]['index'])


  if np.argmax(output_data_q)==np.argmax(Y_test[i]):
    q+=1
  else:
    quantized.append(i)

time9 = time.time()
print('\n\nAccuracy of quantized model= ', q/len(X_test))
print("Time to predict all ", len(X_test), " images is: ", time8-time9)
print("Average time to predict a image is: ", (time9-time8)/len(X_test))
print("Everything done!")

2.4.0
2.4.0
All packages are imported
Test images reading is done
Test images reading is done
Number of Test images 193
Time for reading and Preprocessing 0.593869686126709
Average time for reading and pre-processing 0.0030770450058378706

Time for Loading all models 5.297996282577515
7/7 [==============================] - 1s 60ms/step - loss: 0.5963 - accuracy: 0.9275

Accuracy of Original :  0.9274611473083496
Total time to predict all 0.9594881534576416
Average time to predict 0.0049714412096250864


Accuracy of normal model=  0.9222797927461139
Time to predict all  193  images is:  6.057547092437744
Average time to predict a image is:  0.0313862543649624


Accuracy of quantized model=  0.9222797927461139
Time to predict all  193  images is:  -84.86572670936584
Average time to predict a image is:  0.43971879124023755
Everything done!


**DWT_L6_Reconstruction**

In [8]:
#PUt the paths here.
test_data = '/content/drive/MyDrive/Brain Cancer/Test Data for RPi'
original_model = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Recons_L6.h5'
tflite_model = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Recons_L6_TFLite.tflite'
tflite_model_quantized = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Recons_L6_TFLite_Quantized.tflite'


#Imort the packages
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(tf.keras.__version__)    
#Output must be 
#2.3.0
#2.4.0
import numpy as np
import cv2,pywt
import  os
from sklearn.preprocessing import OneHotEncoder
print('All packages are imported')

def read_image(im_path, level=8):
  img = cv2.imread(im_path,0)
  img = cv2.resize(img,(256,256))
  out = np.zeros((256,256))
  for i in range(0,8):
    row = img.shape[0]
    col = img.shape[1]
    cA,(cH,cV,cD) = pywt.dwt2(img,'haar')
    out[0:(row//2),0:(col//2)] = cA
    out[0:(row//2),(col//2):col] = cH
    out[(row//2):row,0:(col//2)] = cV
    out[(row//2):row,(col//2):col] = cD
    img = cA
    
  #out[0,0] = 0
  out[0:(row//2),0:(col//2)] = 0

  for j in range(0,6):
    row = img.shape[0]
    col = img.shape[1]
    cA = out[0:row,0:col]
    cH = out[0:row,col:2*col]
    cV = out[row:2*row,0:col]
    cD = out[row:2*row,col:2*col]
    img = pywt.idwt2((cA,(cH,cV,cD)),'haar')
    out[0:2*row,0:2*col] = img
    

  return img
  
  
time0 = time.time()
x_test, y_test = [], []
list_folder=os.listdir(path = test_data)
for i in list_folder:
  new_path=os.path.join(test_data,i) 
  pic_list=os.listdir(new_path)                                              
  for img in pic_list:  
    im_path = os.path.join(new_path,img)
    im_dwt = read_image(im_path)
    if i == 'flair':
      label = 0
    elif i == 't1':
      label = 1
    elif i == 't1ce':
      label = 2
    elif i == 't2':
      label = 3
    x_test.append(im_dwt) 
    y_test.append(label) 

print("Test images reading is done")
X_test = np.array(x_test).reshape(-1,64,64,1)
X_test = X_test/255
y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)
Y_test = y_test.toarray()
time1 = time.time()
print("Test images reading is done")
print("Number of Test images",X_test.shape[0])
print("Time for reading and Preprocessing", time1 - time0)
print("Average time for reading and pre-processing", (time1-time0)/X_test.shape[0])


#Read the models
# Initialize TFLite interpreter using the model.
# Load TFLite model and allocate tensors.
time2 = time.time()
#original
original = tf.keras.models.load_model(original_model)
#TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model)
interpreter.allocate_tensors()
#quantized
interpreter_q = tf.lite.Interpreter(model_path=tflite_model_quantized)
interpreter_q.allocate_tensors()
time3 = time.time()
print("\nTime for Loading all models", time3 - time2)


# Get input and output tensors.
#TFLite
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
#Quantized
input_details_q = interpreter_q.get_input_details()
output_details_q = interpreter_q.get_output_details()

#Evaluation
#original
time4 = time.time()
evaluation = original.evaluate(X_test, Y_test)
time5 = time.time()
print('\nAccuracy of Original : ', evaluation[1])
print("Total time to predict all", time5 -time4)
print("Average time to predict", (time5 -time4)/len(X_test))

time6 = time.time()
n = 0  #count for accuracy
normal = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter.set_tensor(input_details[0]['index'], test_image)
  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  
  if np.argmax(output_data)==np.argmax(Y_test[i]):
    n+=1
  else:
    normal.append(i)

time7 = time.time()
print('\n\nAccuracy of normal model= ', n/len(X_test))
print("Time to predict all " , len(X_test), " images is: ", time7-time6)
print("Average time to predict a image is: ", (time7-time6)/len(X_test))
    
time8 = time.time()
q = 0
quantized = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter_q.set_tensor(input_details_q[0]['index'], test_image)
  interpreter_q.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data_q = interpreter_q.get_tensor(output_details_q[0]['index'])


  if np.argmax(output_data_q)==np.argmax(Y_test[i]):
    q+=1
  else:
    quantized.append(i)

time9 = time.time()
print('\n\nAccuracy of quantized model= ', q/len(X_test))
print("Time to predict all ", len(X_test), " images is: ", time9-time8)
print("Average time to predict a image is: ", (time9-time8)/len(X_test))
print("Everything done!")

2.4.0
2.4.0
All packages are imported
Test images reading is done
Test images reading is done
Number of Test images 193
Time for reading and Preprocessing 1.1093683242797852
Average time for reading and pre-processing 0.005748022405594742

Time for Loading all models 6.634419918060303
7/7 [==============================] - 0s 13ms/step - loss: 0.2709 - accuracy: 0.9171

Accuracy of Original :  0.9170984625816345
Total time to predict all 0.31792163848876953
Average time to predict 0.0016472623755894795


Accuracy of normal model=  0.9119170984455959
Time to predict all  193  images is:  0.6785969734191895
Average time to predict a image is:  0.0035160464943999456


Accuracy of quantized model=  0.9119170984455959
Time to predict all  193  images is:  6.576596736907959
Average time to predict a image is:  0.034075630761181135
Everything done!


**DWT + Spark**

In [9]:
#PUt the paths here.
test_data = '/content/drive/MyDrive/Brain Cancer/Test Data for RPi'
original_model = '/content/drive/MyDrive/Brain Cancer/v3/dwt_spark.h5'
tflite_model = '/content/drive/MyDrive/Brain Cancer/v3/dwt_spark_tflite.tflite'
tflite_model_quantized = '/content/drive/MyDrive/Brain Cancer/v3/dwt_spark_quantized.tflite'


#Imort the packages
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(tf.keras.__version__)    
#Output must be 
#2.3.0
#2.4.0
import numpy as np
import cv2
import  os
from sklearn.preprocessing import OneHotEncoder
import pywt
print('All packages are imported')


def read_image(im_path):
  img = cv2.imread(im_path,0)
  img = cv2.resize(img,(240,240))
  return img
  
  
time0 = time.time()
x_test, y_test = [], []
list_folder=os.listdir(path = test_data)
for i in list_folder:
  new_path=os.path.join(test_data,i) 
  pic_list=os.listdir(new_path)                                              
  for img in pic_list:  
    im_path = os.path.join(new_path,img)
    image = read_image(im_path)
    cA, _ = pywt.dwt2(image,'db2')    #Performing DWT operation
    if i == 'flair':
      label = 0
    elif i == 't1':
      label = 1
    elif i == 't1ce':
      label = 2
    elif i == 't2':
      label = 3
    x_test.append(cA) 
    y_test.append(label) 


X_test = np.array(x_test).reshape(-1,121,121,1)
X_test = X_test/255
y_test = np.array(y_test).reshape(-1,1)
encoder = OneHotEncoder()
y_test = encoder.fit_transform(y_test)
Y_test = y_test.toarray()
time1 = time.time()
print("Test images reading is done")
print("Number of Test images",X_test.shape[0])
print("Time for reading and Preprocessing", time1 - time0)
print("Average time for reading and pre-processing", (time1-time0)/X_test.shape[0])


#Read the models
# Initialize TFLite interpreter using the model.
# Load TFLite model and allocate tensors.
time2 = time.time()
#original
original = tf.keras.models.load_model(original_model)
#TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model)
interpreter.allocate_tensors()
#quantized
interpreter_q = tf.lite.Interpreter(model_path=tflite_model_quantized)
interpreter_q.allocate_tensors()
time3 = time.time()
print("\nTime for Loading all models", time3 - time2)


# Get input and output tensors.
#TFLite
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
#Quantized
input_details_q = interpreter_q.get_input_details()
output_details_q = interpreter_q.get_output_details()

#Evaluation
#original
time4 = time.time()
evaluation = original.evaluate(X_test, Y_test)
time5 = time.time()
print('\Accuracy of Original : ', evaluation[1])
print("Total time to predict all", time5 -time4)
print("Average time to predict: ", (time5 -time4)/len(X_test))

time6 = time.time()
n = 0  #count for accuracy
normal = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter_q.set_tensor(input_details_q[0]['index'], test_image)
  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  
  if np.argmax(output_data)==np.argmax(Y_test[i]):
    n+=1
  else:
    normal.append(i)

time7 = time.time()
print('\n\nAccuracy of normal model= ', n/len(X_test))
print("Time to predict all " , len(X_test), " images is: ", time7-time6)
print("Average time to predict a image is: ", (time7-time6)/len(X_test))
    
time8 = time.time()
q = 0
quantized = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter_q.set_tensor(input_details_q[0]['index'], test_image)
  interpreter_q.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data_q = interpreter_q.get_tensor(output_details_q[0]['index'])


  if np.argmax(output_data_q)==np.argmax(Y_test[i]):
    q+=1
  else:
    quantized.append(i)

time9 = time.time()
print('\n\nAccuracy of quantized model= ', q/len(X_test))
print("Time to predict all ", len(X_test), " images is: ", time9-time8)
print("Average time to predict a image is: ", (time9-time8)/len(X_test))


2.4.0
2.4.0
All packages are imported
Test images reading is done
Number of Test images 193
Time for reading and Preprocessing 0.8160793781280518
Average time for reading and pre-processing 0.0042283905602489725

Time for Loading all models 1.5825834274291992
7/7 [==============================] - 0s 16ms/step - loss: 0.1102 - accuracy: 0.9637
\Accuracy of Original :  0.9637305736541748
Total time to predict all 0.39378809928894043
Average time to predict:  0.002040352846056686


Accuracy of normal model=  0.24352331606217617
Time to predict all  193  images is:  0.7238483428955078
Average time to predict a image is:  0.0037505095486813876


Accuracy of quantized model=  0.9585492227979274
Time to predict all  193  images is:  3.6732611656188965
Average time to predict a image is:  0.019032441272636768


**DWT Reconstruction L7**

In [10]:
#PUt the paths here.
test_data = '/content/drive/MyDrive/Brain Cancer/Test Data for RPi'
original_model = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Recons_L7.h5'
tflite_model = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Recons_L7_TFLite.tflite'
tflite_model_quantized = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Recons_L7_TFLite_Quantized.tflite'


#Imort the packages
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(tf.keras.__version__)    
#Output must be 
#2.3.0
#2.4.0
import numpy as np
import cv2
import  os
from sklearn.preprocessing import OneHotEncoder
import pywt
print('All packages are imported')


def read_image(im_path, level=8):
  img = cv2.imread(im_path,0)
  img = cv2.resize(img,(256,256))
  out = np.zeros((256,256))
  for i in range(0,8):
    row = img.shape[0]
    col = img.shape[1]
    cA,(cH,cV,cD) = pywt.dwt2(img,'haar')
    out[0:(row//2),0:(col//2)] = cA
    out[0:(row//2),(col//2):col] = cH
    out[(row//2):row,0:(col//2)] = cV
    out[(row//2):row,(col//2):col] = cD
    img = cA
    
  #out[0,0] = 0
  out[0:(row//2),0:(col//2)] = 0

  for j in range(0,7):
    row = img.shape[0]
    col = img.shape[1]
    cA = out[0:row,0:col]
    cH = out[0:row,col:2*col]
    cV = out[row:2*row,0:col]
    cD = out[row:2*row,col:2*col]
    img = pywt.idwt2((cA,(cH,cV,cD)),'haar')
    out[0:2*row,0:2*col] = img
    

  return img


time0 = time.time()
x_test, y_test = [], []
list_folder=os.listdir(path = test_data)
for i in list_folder:
  new_path=os.path.join(test_data,i) 
  pic_list=os.listdir(new_path)                                              
  for img in pic_list:  
    im_path = os.path.join(new_path,img)
    im_dwt = read_image(im_path)
    if i == 'flair':
      label = 0
    elif i == 't1':
      label = 1
    elif i == 't1ce':
      label = 2
    elif i == 't2':
      label = 3
    x_test.append(im_dwt) 
    y_test.append(label) 

print("Test images reading is done")
X_test = np.array(x_test).reshape(-1,128,128,1)
X_test = X_test/255
y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)
Y_test = y_test.toarray()
time1 = time.time()
print("Test images reading is done")
print("Number of Test images",X_test.shape[0])
print("Time for reading and Preprocessing", time1 - time0)
print("Average time for reading and pre-processing", (time1-time0)/X_test.shape[0])


#Read the models
# Initialize TFLite interpreter using the model.
# Load TFLite model and allocate tensors.
time2 = time.time()
#original
original = tf.keras.models.load_model(original_model)
#TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model)
interpreter.allocate_tensors()
#quantized
interpreter_q = tf.lite.Interpreter(model_path=tflite_model_quantized)
interpreter_q.allocate_tensors()
time3 = time.time()
print("\nTime for Loading all models", time3 - time2)


# Get input and output tensors.
#TFLite
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
#Quantized
input_details_q = interpreter_q.get_input_details()
output_details_q = interpreter_q.get_output_details()

#Evaluation
#original
time4 = time.time()
evaluation = original.evaluate(X_test, Y_test)
time5 = time.time()
print('\nAccuracy of Original : ', evaluation[1])
print("Total time to predict all", time5 -time4)
print("Average time to predict", (time5 -time4)/len(X_test))

time6 = time.time()
n = 0  #count for accuracy
normal = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter.set_tensor(input_details[0]['index'], test_image)
  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  
  if np.argmax(output_data)==np.argmax(Y_test[i]):
    n+=1
  else:
    normal.append(i)

time7 = time.time()
print('\n\nAccuracy of TFLite model= ', n/len(X_test))
print("Time to predict all " , len(X_test), " images is: ", time7-time6)
print("Average time to predict a image is: ", (time7-time6)/len(X_test))
    
time8 = time.time()
q = 0
quantized = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter_q.set_tensor(input_details_q[0]['index'], test_image)
  interpreter_q.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data_q = interpreter_q.get_tensor(output_details_q[0]['index'])


  if np.argmax(output_data_q)==np.argmax(Y_test[i]):
    q+=1
  else:
    quantized.append(i)

time9 = time.time()
print('\n\nAccuracy of quantized model= ', q/len(X_test))
print("Time to predict all ", len(X_test), " images is: ", time9-time8)
print("Average time to predict a image is: ", (time9-time8)/len(X_test))
print("Everything done!")

2.4.0
2.4.0
All packages are imported
Test images reading is done
Test images reading is done
Number of Test images 193
Time for reading and Preprocessing 1.1251811981201172
Average time for reading and pre-processing 0.005829954394404752

Time for Loading all models 11.373985290527344
7/7 [==============================] - 0s 13ms/step - loss: 0.1753 - accuracy: 0.9430

Accuracy of Original :  0.9430052042007446
Total time to predict all 0.30805468559265137
Average time to predict 0.0015961382673194372


Accuracy of TFLite model=  0.9378238341968912
Time to predict all  193  images is:  1.756357192993164
Average time to predict a image is:  0.009100296336752146


Accuracy of quantized model=  0.9430051813471503
Time to predict all  193  images is:  69.2361581325531
Average time to predict a image is:  0.35873657063499015
Everything done!


**DWT Reconstruction**

In [ ]:
#PUt the paths here.
test_data = '/content/drive/MyDrive/Brain Cancer/Test Data for RPi'
original_model = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Reconstruction.h5'
tflite_model = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Reconstruction_TFLite.tflite'
tflite_model_quantized = '/content/drive/MyDrive/Brain Cancer/v3/DWT_Reconstruction_TFLite_Quantized.tflite'


#Imort the packages
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(tf.keras.__version__)    
#Output must be 
#2.3.0
#2.4.0
import numpy as np
import cv2
import  os
from sklearn.preprocessing import OneHotEncoder
import pywt
print('All packages are imported')


def read_image(im_path, level=8):
  img = cv2.imread(im_path,0)
  img = cv2.resize(img,(256,256))
  out = np.zeros((256,256))
  for i in range(0,8):
    row = img.shape[0]
    col = img.shape[1]
    cA,(cH,cV,cD) = pywt.dwt2(img,'haar')
    out[0:(row//2),0:(col//2)] = cA
    out[0:(row//2),(col//2):col] = cH
    out[(row//2):row,0:(col//2)] = cV
    out[(row//2):row,(col//2):col] = cD
    img = cA
    
  #out[0,0] = 0
  out[0:(row//2),0:(col//2)] = 0

  for j in range(0,8):
    row = img.shape[0]
    col = img.shape[1]
    cA = out[0:row,0:col]
    cH = out[0:row,col:2*col]
    cV = out[row:2*row,0:col]
    cD = out[row:2*row,col:2*col]
    img = pywt.idwt2((cA,(cH,cV,cD)),'haar')
    out[0:2*row,0:2*col] = img
    

  return img


time0 = time.time()
x_test, y_test = [], []
list_folder=os.listdir(path = test_data)
for i in list_folder:
  new_path=os.path.join(test_data,i) 
  pic_list=os.listdir(new_path)                                              
  for img in pic_list:  
    im_path = os.path.join(new_path,img)
    im_dwt = read_image(im_path)
    if i == 'flair':
      label = 0
    elif i == 't1':
      label = 1
    elif i == 't1ce':
      label = 2
    elif i == 't2':
      label = 3
    x_test.append(im_dwt) 
    y_test.append(label) 

print("Test images reading is done")
X_test = np.array(x_test).reshape(-1,256,256,1)
X_test = X_test/255
y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)
Y_test = y_test.toarray()
time1 = time.time()
print("Test images reading is done")
print("Number of Test images",X_test.shape[0])
print("Time for reading and Preprocessing", time1 - time0)
print("Average time for reading and pre-processing", (time1-time0)/X_test.shape[0])


#Read the models
# Initialize TFLite interpreter using the model.
# Load TFLite model and allocate tensors.
time2 = time.time()
#original
original = tf.keras.models.load_model(original_model)
#TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model)
interpreter.allocate_tensors()
#quantized
interpreter_q = tf.lite.Interpreter(model_path=tflite_model_quantized)
interpreter_q.allocate_tensors()
time3 = time.time()
print("\nTime for Loading all models", time3 - time2)


# Get input and output tensors.
#TFLite
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
#Quantized
input_details_q = interpreter_q.get_input_details()
output_details_q = interpreter_q.get_output_details()

#Evaluation
#original
time4 = time.time()
evaluation = original.evaluate(X_test, Y_test)
time5 = time.time()
print('\nAccuracy of Original : ', evaluation[1])
print("Total time to predict all", time5 -time4)
print("Average time to predict", (time5 -time4)/len(X_test))

time6 = time.time()
n = 0  #count for accuracy
normal = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter.set_tensor(input_details[0]['index'], test_image)
  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  
  if np.argmax(output_data)==np.argmax(Y_test[i]):
    n+=1
  else:
    normal.append(i)

time7 = time.time()
print('\n\nAccuracy of TFLite model= ', n/len(X_test))
print("Time to predict all " , len(X_test), " images is: ", time7-time6)
print("Average time to predict a image is: ", (time7-time6)/len(X_test))
    
time8 = time.time()
q = 0
quantized = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter_q.set_tensor(input_details_q[0]['index'], test_image)
  interpreter_q.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data_q = interpreter_q.get_tensor(output_details_q[0]['index'])


  if np.argmax(output_data_q)==np.argmax(Y_test[i]):
    q+=1
  else:
    quantized.append(i)

time9 = time.time()
print('\n\nAccuracy of quantized model= ', q/len(X_test))
print("Time to predict all ", len(X_test), " images is: ", time9-time8)
print("Average time to predict a image is: ", (time9-time8)/len(X_test))
print("Everything done!")

**Normal**

In [ ]:
#PUt the paths here.
test_data = '/content/drive/MyDrive/Brain Cancer/Test Data for RPi'
original_model = '/content/drive/MyDrive/Brain Cancer/v2/Normal_tensorflow.h5'
tflite_model = '/content/drive/MyDrive/Brain Cancer/v2/Normal_TFLite.tflite'
tflite_model_quantized = '/content/drive/MyDrive/Brain Cancer/v2/Normal_Quantized_TFLite.tflite'


#Imort the packages
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(tf.keras.__version__)    
#Output must be 
#2.3.0
#2.4.0
import numpy as np
import cv2
import  os
from sklearn.preprocessing import OneHotEncoder
print('All packages are imported')

def read_image(im_path):
  img = cv2.imread(im_path)
  img = cv2.resize(img,(240,240))
  return img

x_test, y_test = [], []
list_folder=os.listdir(path = test_data)
for i in list_folder:
  new_path=os.path.join(test_data,i) 
  pic_list=os.listdir(new_path)                                              
  for img in pic_list:  
    im_path = os.path.join(new_path,img)
    image = read_image(im_path)
    if i == 'flair':
      label = 0
    elif i == 't1':
      label = 1
    elif i == 't1ce':
      label = 2
    elif i == 't2':
      label = 3
    x_test.append(image) 
    y_test.append(label) 

encoder = OneHotEncoder()
print("Test images reading is done")
X_test = np.array(x_test).reshape(-1,240,240,3)
X_test = X_test/255
y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)
Y_test = y_test.toarray()
time1 = time.time()
print("Test images reading is done")
print("Number of Test images",X_test.shape[0])
print("Time for reading and Preprocessing", time1 - time0)
print("Average time for reading and pre-processing", (time1-time0)/X_test.shape[0])


#Read the models
# Initialize TFLite interpreter using the model.
# Load TFLite model and allocate tensors.
time2 = time.time()
#original
original = tf.keras.models.load_model(original_model)
#TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model)
interpreter.allocate_tensors()
#quantized
interpreter_q = tf.lite.Interpreter(model_path=tflite_model_quantized)
interpreter_q.allocate_tensors()
time3 = time.time()
print("\nTime for Loading all models", time3 - time2)


# Get input and output tensors.
#TFLite
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
#Quantized
input_details_q = interpreter_q.get_input_details()
output_details_q = interpreter_q.get_output_details()

#Evaluation
#original
time4 = time.time()
evaluation = original.evaluate(X_test, Y_test)
time5 = time.time()
print('\nAccuracy of Original : ', evaluation[1])
print("Total time to predict all", time5 -time4)
print("Average time to predict", (time5 -time4)/len(X_test))

time6 = time.time()
n = 0  #count for accuracy
normal = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter.set_tensor(input_details[0]['index'], test_image)
  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  
  if np.argmax(output_data)==np.argmax(Y_test[i]):
    n+=1
  else:
    normal.append(i)

time7 = time.time()
print('\n\nAccuracy of TFLite model= ', n/len(X_test))
print("Time to predict all " , len(X_test), " images is: ", time7-time6)
print("Average time to predict a image is: ", (time7-time6)/len(X_test))
    
time8 = time.time()
q = 0
quantized = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter_q.set_tensor(input_details_q[0]['index'], test_image)
  interpreter_q.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data_q = interpreter_q.get_tensor(output_details_q[0]['index'])


  if np.argmax(output_data_q)==np.argmax(Y_test[i]):
    q+=1
  else:
    quantized.append(i)

time9 = time.time()
print('\n\nAccuracy of quantized model= ', q/len(X_test))
print("Time to predict all ", len(X_test), " images is: ", time9-time8)
print("Average time to predict a image is: ", (time9-time8)/len(X_test))
print("Everything done!")

**DWT**

In [ ]:
#PUt the paths here.
test_data = '/content/drive/MyDrive/Brain Cancer/Test Data for RPi'
original_model = '/content/drive/MyDrive/Brain Cancer/v2/dwt_normal.h5'
tflite_model = '/content/drive/MyDrive/Brain Cancer/v2/dwt_tflite.tflite'
tflite_model_quantized = '/content/drive/MyDrive/Brain Cancer/v2/dwt_quantized_TFLite.tflite'


#Imort the packages
import time
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(tf.keras.__version__)    
#Output must be 
#2.3.0
#2.4.0
import numpy as np
import cv2,pywt
import  os
from sklearn.preprocessing import OneHotEncoder
print('All packages are imported')

def read_image(im_path):
  img = cv2.imread(im_path)
  img = cv2.resize(img,(240,240))
  return img

x_test, y_test = [], []
list_folder=os.listdir(path = test_data)
for i in list_folder:
  new_path=os.path.join(test_data,i) 
  pic_list=os.listdir(new_path)                                              
  for img in pic_list:  
    im_path = os.path.join(new_path,img)
    image = read_image(im_path)
    if i == 'flair':
      label = 0
    elif i == 't1':
      label = 1
    elif i == 't1ce':
      label = 2
    elif i == 't2':
      label = 3
    x_test.append(image) 
    y_test.append(label) 

encoder = OneHotEncoder()
print("Test images reading is done")
X_test = np.array(x_test).reshape(-1,240,240,3)
X_test = X_test/255
y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)
Y_test = y_test.toarray()
time1 = time.time()
print("Test images reading is done")
print("Number of Test images",X_test.shape[0])
print("Time for reading and Preprocessing", time1 - time0)
print("Average time for reading and pre-processing", (time1-time0)/X_test.shape[0])


#Read the models
# Initialize TFLite interpreter using the model.
# Load TFLite model and allocate tensors.
time2 = time.time()
#original
original = tf.keras.models.load_model(original_model)
#TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model)
interpreter.allocate_tensors()
#quantized
interpreter_q = tf.lite.Interpreter(model_path=tflite_model_quantized)
interpreter_q.allocate_tensors()
time3 = time.time()
print("\nTime for Loading all models", time3 - time2)


# Get input and output tensors.
#TFLite
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
#Quantized
input_details_q = interpreter_q.get_input_details()
output_details_q = interpreter_q.get_output_details()

#Evaluation
#original
time4 = time.time()
evaluation = original.evaluate(X_test, Y_test)
time5 = time.time()
print('\nAccuracy of Original : ', evaluation[1])
print("Total time to predict all", time5 -time4)
print("Average time to predict", (time5 -time4)/len(X_test))

time6 = time.time()
n = 0  #count for accuracy
normal = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter.set_tensor(input_details[0]['index'], test_image)
  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data = interpreter.get_tensor(output_details[0]['index'])
  
  if np.argmax(output_data)==np.argmax(Y_test[i]):
    n+=1
  else:
    normal.append(i)

time7 = time.time()
print('\n\nAccuracy of TFLite model= ', n/len(X_test))
print("Time to predict all " , len(X_test), " images is: ", time7-time6)
print("Average time to predict a image is: ", (time7-time6)/len(X_test))
    
time8 = time.time()
q = 0
quantized = []
# Test model on input data.
for i in range(1, len(X_test)):
  test_image = np.expand_dims(X_test[i], axis=0).astype(np.float32)
  interpreter_q.set_tensor(input_details_q[0]['index'], test_image)
  interpreter_q.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.
  output_data_q = interpreter_q.get_tensor(output_details_q[0]['index'])


  if np.argmax(output_data_q)==np.argmax(Y_test[i]):
    q+=1
  else:
    quantized.append(i)

time9 = time.time()
print('\n\nAccuracy of quantized model= ', q/len(X_test))
print("Time to predict all ", len(X_test), " images is: ", time9-time8)
print("Average time to predict a image is: ", (time9-time8)/len(X_test))
print("Everything done!")